In [34]:
from azure.identity import DefaultAzureCredential

account_url = f"https://wboylesbackups.blob.core.windows.net"
credential = DefaultAzureCredential()

In [ ]:
from azure.storage.blob import BlobClient

def download_blob(container_name: str, blob_name: str, out_filename: str | None = None):
    if out_filename is None:
        out_filename = container_name + blob_name

    blob_client = BlobClient(account_url, container_name, blob_name, credential=credential)

    with open(out_filename, "wb") as f:
        f.write(blob_client.download_blob().readall())

In [70]:
from azure.storage.blob import BlobPrefix, ContainerClient, BlobProperties

# Returns list of BlobProperties in preorder traversal
# Does not include folders, only files
def list_blobs(container_name: str, prefix: str="") -> list[BlobProperties]:
    blobs: list[BlobProperties] = []
    container_client = ContainerClient(account_url, container_name, credential)

    for blob in container_client.walk_blobs(name_starts_with=prefix):
        name: str = blob.name # type: ignore

        if isinstance(blob, BlobPrefix):
            blobs += list_blobs(container_name, prefix=name)
        else:
            blobs.append(blob)


    return blobs